# Setup Python Environment
This cell installs the required Python packages (`rpy2`) for running R code in this notebook.

In [ ]:
import sys
try:
    import rpy2
except ImportError:
    !{sys.executable} -m pip install rpy2

# Load Python Environment
This cell loads the required Python packages (`rpy2`) for running R code in this notebook.

In [6]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


# Load Libraries in R

In [ ]:
%%R
library(readxl)
library(tidyverse)

dataset <- read_excel("data/personal_finance_dataset.xlsx", sheet = "datathon_finance")
print(as.data.frame(head(dataset, 20), width = Inf))

# [shelly put stuff here]
Performing forward stepwise selection to determine the best model to predict the net worth of an individual.